In [2]:
import glob
import os
from pathlib import Path

# https://github.com/chezou/tabula-py
# https://anaconda.org/conda-forge/tabula-py
import tabula
import pandas as pd
import numpy as np

In [205]:
# Custom mappings between tabula imports and actual column names.
# Mappings with duplicate columns are split on spaces.
col_map_2009 = {
    'Senate Senate': ['senate_dem', 'senate_rep'],
    'Total House': ['house_total', 'house_dem'],
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Unnamed: 1': 'senate_other',
    'House': 'house_rep',
    'Unnamed: 2': 'house_other'
}
col_map_2010 = {
    'Senate Senate': ['senate_dem', 'senate_rep'],
    'House House': ['house_dem', 'house_rep'],
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Unnamed: 1': 'senate_other',
    'Total.2': 'house_total',
    'Unnamed: 2': 'house_other'
}
col_map_2011_2012 = {
    'Senate Senate': ['senate_dem', 'senate_rep'],
    'House House': ['house_dem', 'house_rep'],
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Unnamed: 1': 'senate_other',
    'Total.2': 'house_total',
    'House': 'house_other'   
}
col_map_2014 = {
    'Senate Senate': ['senate_dem', 'senate_rep'],
    'House House': ['house_dem', 'house_rep'],
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Unnamed: 1': 'senate_other',
    'Total.2': 'house_total',
    'House': 'house_other',
}
col_map_2015 = {
    'Senate.1': ['senate_dem', 'senate_rep'],
    'House House': ['house_dem', 'house_rep'],
    'Total Total': ['total', 'senate_total'],
    'Unnamed: 0': 'state',
    'Senate.2': 'senate_other',
    'Total': 'house_total',
    'House': 'house_other',
}
col_map_2016 = {
    'Total Total': ['total', 'senate_total'],
    'Senate Senate': ['senate_dem', 'senate_rep'],
    'House House': ['house_dem', 'house_rep'],
    'Unnamed: 0': 'state',
    'Senate': 'senate_other',
    'Total': 'house_total',
    'House': 'house_other',
}
col_map_2017_2020 = {
    'Total House': ['house_total', 'house_dem'],
    'House': 'house_rep',
    'House.1': 'house_other',
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Senate': 'senate_dem',
    'Senate.1': 'senate_rep',
    'Senate.2': 'senate_other'
}
col_map_2021 = {
    'Unnamed: 0': 'state',
    'Total.1': 'senate_total',
    'Senate': 'senate_dem',
    'Senate.1': 'senate_rep',
    'Senate Total': ['senate_other', 'house_total'],
    'House': 'house_dem',
    'House.1': 'house_rep',
    'House.2': 'house_other'
}
col_map = {
    'Legis_Control_2009': col_map_2009,
    'Legis_Control_2010': col_map_2010,
    'Legis_Control_2011': col_map_2011_2012,
    'Legis_Control_2012': col_map_2011_2012,
    'Legis_Control_2014': col_map_2014,
    'Legis_Control_2015': col_map_2015,
    'Legis_Control_2016': col_map_2016,
    'Legis_Control_2017': col_map_2017_2020,
    'Legis_Control_2018': col_map_2017_2020,
    'Legis_Control_2019': col_map_2017_2020,
    'Legis_Control_2020': col_map_2017_2020,
    'Legis_Control_2021': col_map_2021   
}

def import_pdfs():
    tabula_dfs = {}
    p = Path('./data/inputs')
    for f in sorted(p.glob('*.pdf')):
        print(f.name)
        tabula_dfs[f.stem] = tabula.read_pdf(f.resolve(), pages=1)[0] # First table
    return tabula_dfs

def split_column(df, col, names):
    try:
        df = df.copy()
        df[names] = df[col].str.split(' ', expand=True) #Split all
    except Exception as e:
        print(df[col].str.split(' ', expand=True))
    return df

def process_tabula(df, column_mapping, year, drop_rows=[0]):
    df = df.copy()
    df = df.drop(drop_rows)
    df.reset_index(drop=True, inplace=True)
    wy_index = df.index[df['Unnamed: 0'] == 'Wyoming'].tolist()[0]
    df = df.iloc[0:wy_index+1]
    
    # Handle 2017-2021 Unicameral syntax. May work for 2013 too, although
    # I don't believe it matches within cell strings, just entire cells.
    df.replace(
        ['Unicameral - nonpartisan', '4U9nicameral - nonpartisan', '1, 1v'],
        ['Unicameral', 'Unicameral', '1,1v'],
        inplace=True
    )
    
    for key, value in column_mapping.items():
        if type(value) == list:
            df = split_column(df, col=key, names=value)
        else:
            df = df.rename(columns={key:value})
            
    cols = [
        'state', 'senate_total', 'senate_dem', 'senate_rep', 'senate_other',
        'house_total', 'house_dem', 'house_rep', 'house_other'
    ]
    df = df[cols]
    df['year'] = year
    return df

def process_2013(df_2013):
    # 2013 is a messed up import, needs to be custom.
    df_2013 = df_2013.copy() # Could modify in place, but return instead.
    df_2013 = df_2013.drop([0,1])
    df_2013.reset_index(drop=True, inplace=True)
    wy_index = df_2013.index[df_2013['Unnamed: 0'] == 'Wyoming'].tolist()[0]
    df_2013 = df_2013.iloc[0:wy_index+1]
    df_2013['2013 State and Legislative Partisan Composition'] = df_2013['2013 State and Legislative Partisan Composition'].str.replace('1, 1v', '1,1v', regex=False)
    df_2013['2013 State and Legislative Partisan Composition'] = df_2013['2013 State and Legislative Partisan Composition'].str.replace('1, 1V', '1,1v', regex=False)
    split_2013 = df_2013['2013 State and Legislative Partisan Composition'].str.split(' ', expand=True)
    df_2013 = df_2013['Unnamed: 0'].to_frame().join(split_2013)
    df_2013.rename(
        columns = {
            'Unnamed: 0': 'state',
            1: 'senate_total',
            2: 'senate_dem', 
            3: 'senate_rep',
            4: 'senate_other', 
            5: 'house_total',
            6: 'house_dem',
            7: 'house_rep',
            8: 'house_other'
        },
        inplace=True
    )
    df_2013 = df_2013[[
        'state', 'senate_total', 'senate_dem', 'senate_rep', 'senate_other',
        'house_total', 'house_dem', 'house_rep', 'house_other'
    ]]
    df_2013['year'] = 2013
    return df_2013
    
def process_all_years(tabula_dfs):
    
    results = {}
    
    for key, df in tabula_dfs.items():
        if key in ['Legis_Control_2013', 'Legis_Control_2021']:
            pass
        else:
            print(key)
            results[key] = process_tabula(
                df=df,
                column_mapping=col_map[key],
                year=int(key[-4:])
            )
    # Each processed tabula df is in dict, so concat at end
    #results_df = pd.concat(results.values())
    return results

In [183]:
tabula_dfs = import_pdfs()

Legis_Control_2009.pdf
Legis_Control_2010.pdf
Legis_Control_2011.pdf
Legis_Control_2012.pdf
Legis_Control_2013.pdf
Legis_Control_2014.pdf
Legis_Control_2015.pdf
Legis_Control_2016.pdf
Legis_Control_2017.pdf
Legis_Control_2018.pdf
Legis_Control_2019.pdf
Legis_Control_2020.pdf
Legis_Control_2021.pdf


In [186]:
# Imported columns are barely uniform across any of the years. Tough to automate
# any of this, although it looks like 2011, 2012 are same, and 2017-2020 are as well.
# 2013 is one large column.
ls = []
for key, value in tabula_dfs.items():
    ls.append([key] + list(value.columns))
col_df = pd.DataFrame(ls) #columns = ['Name', 'Age']
col_df['duplicated'] = col_df.duplicated(keep=False, subset=list(col_df.columns)[1:])
col_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,duplicated
0,Legis_Control_2009,Unnamed: 0,Total,Total.1,Senate Senate,Senate,Unnamed: 1,Total House,House,House.1,Unnamed: 2,Legis.,Gov.,State,None,False
1,Legis_Control_2010,Unnamed: 0,Total,Total.1,Senate Senate,Senate,Unnamed: 1,Total.2,House House,House,Unnamed: 2,Legis.,Gov.,State,None,False
2,Legis_Control_2011,Unnamed: 0,Total,Total.1,Senate Senate,Senate,Unnamed: 1,Total.2,House House,Unnamed: 2,House,Legis.,Gov.,State,None,True
3,Legis_Control_2012,Unnamed: 0,Total,Total.1,Senate Senate,Senate,Unnamed: 1,Total.2,House House,Unnamed: 2,House,Legis.,Gov.,State,None,True
4,Legis_Control_2013,Unnamed: 0,2013 State and Legislative Partisan Composition,"(As of January 31, 2013)",None,None,None,None,None,None,None,None,None,None,None,False
5,Legis_Control_2014,Unnamed: 0,Total,Total.1,Senate Senate,Senate,Unnamed: 1,Total.2,House House,Unnamed: 2,House,Legis.,Unnamed: 3,Gov.,State,False
6,Legis_Control_2015,Unnamed: 0,Total Total,Senate,Senate.1,Senate.2,Total,House House,Unnamed: 1,House,Legis.,Gov.,Unnamed: 2,State,None,False
7,Legis_Control_2016,Unnamed: 0,Total Total,Unnamed: 1,Senate Senate,Senate,Total,House House,Unnamed: 2,House,Unnamed: 3,Legis. Gov.,Unnamed: 4,State,None,False
8,Legis_Control_2017,Unnamed: 0,Total,Total.1,Senate,Senate.1,Senate.2,Total House,House,Unnamed: 1,House.1,Legis. Gov.,Unnamed: 2,State,None,True
9,Legis_Control_2018,Unnamed: 0,Total,Total.1,Senate,Senate.1,Senate.2,Total House,House,Unnamed: 1,House.1,Legis. Gov.,Unnamed: 2,State,None,True


In [267]:
results = process_all_years(tabula_dfs)
# Handle 2013 and 2021 separately
results['Legis_Control_2013'] = process_2013(df_2013=tabula_dfs['Legis_Control_2013'])
# Manually input 2021 because one combined column was inseparable
results['Legis_Control_2021'] = pd.read_csv('./data/inputs/Legis_Control_2021.csv')
results = pd.concat(results.values())
col_order = [
    'state', 'year', 'senate_total', 'senate_dem', 'senate_rep', 'senate_other',
    'house_total', 'house_dem', 'house_rep', 'house_other'
]
results = results[col_order]
results = results.sort_values(by=['year', 'state'], ascending=True)
# Remove occasional asterisk
results['state'] = results['state'].str.replace('*', '', regex=False) 
results

Legis_Control_2009
Legis_Control_2010
Legis_Control_2011
Legis_Control_2012
Legis_Control_2014
Legis_Control_2015
Legis_Control_2016
Legis_Control_2017
Legis_Control_2018
Legis_Control_2019
Legis_Control_2020


,state,year,senate_total,senate_dem,senate_rep,senate_other,house_total,house_dem,house_rep,house_other
0,Alabama,2009,35,19,13,3v,105,62,43,0
1,Alaska,2009,20,10,10,0,40,18,22,0
2,Arizona,2009,30,12,18,0,60,25,35,0
3,Arkansas,2009,35,27,8,0,100,71,28,1
4,California,2009,40,26,14,0,80,51,29,0
...,...,...,...,...,...,...,...,...,...,...
44,Virginia,2021,40,21.0,18.0,1v,100,55.0,45.0,NaN
46,Washington,2021,49,29.0,20.0,NaN,98,57.0,41.0,NaN
48,West Virginia,2021,34,11.0,23.0,NaN,100,23.0,77.0,NaN
47,Wisconsin,2021,33,12.0,20.0,1v,99,38.0,60.0,1v


In [7]:
# Try replacing the "1v" vacant syntax, it's redundant.
# But if it's truly an "other" seat, leave it
results = results.replace({',': ' '}, regex=True)
results[['senate_other', 'house_other']] = results[
    ['senate_other', 'house_other']
].replace({'.[a-z]': ''}, regex=True)
results.replace(r'^\s*$', np.nan, regex=True, inplace=True)
results.replace('Unicameral', 0, regex=True, inplace=True)
results.fillna(0, inplace=True)
int_cols = list(results)
int_cols.remove('state')
results[int_cols] = results[int_cols].astype(float) # First convert float strings to float
results[int_cols] = results[int_cols].astype(int) # Then to ints
results

,state,year,senate_total,senate_dem,senate_rep,senate_other,house_total,house_dem,house_rep,house_other
0,Alabama,2009,35,19,13,0,105,62,43,0
1,Alaska,2009,20,10,10,0,40,18,22,0
2,Arizona,2009,30,12,18,0,60,25,35,0
3,Arkansas,2009,35,27,8,0,100,71,28,1
4,California,2009,40,26,14,0,80,51,29,0
...,...,...,...,...,...,...,...,...,...,...
645,Virginia,2021,40,21,18,0,100,55,45,0
646,Washington,2021,49,29,20,0,98,57,41,0
647,West Virginia,2021,34,11,23,0,100,23,77,0
648,Wisconsin,2021,33,12,20,0,99,38,60,0


In [8]:
results.to_csv('./data/outputs/state_legislative_control_2009_2021.csv', index=False)